In [7]:
period_times = {
    'Period 1 Odd' : {
        'period' : 'Period 1',
        'start_time' : '8:20 AM',
        'end_time' : '9:35 AM',
    },
    'Period 2 Odd' : {
        'period' : 'Period 2',
        'start_time' : '9:55 AM',
        'end_time' : '10:55 AM',
    },
    'Period 3 Odd' : {
        'period' : 'Period 3',
        'start_time' : '11:00 AM',
        'end_time' : '12:00 PM',
    },
    'Period 4 Odd' : {
        'period' : 'Period 4',
        'start_time' : '1:05 PM',
        'end_time' : '2:05 PM',
    },
    'Period 5 Odd' : {
        'period' : 'Period 5',
        'start_time' : '2:10 PM',
        'end_time' : '3:10 PM',
    },

    'Period 1 Even' : {
        'period' : 'Period 1',
        'start_time' : '8:20 AM',
        'end_time' : '9:20 AM',
    },
    'Period 2 Even' : {
        'period' : 'Period 2',
        'start_time' : '9:55 AM',
        'end_time' : '10:55 AM',
    },
    'Period 3 Even' : {
        'period' : 'Period 3',
        'start_time' : '11:00 AM',
        'end_time' : '12:15 PM',
    },
    'Period 4 Even' : {
        'period' : 'Period 4',
        'start_time' : '1:05 PM',
        'end_time' : '2:05 PM',
    },
    'Period 5 Even' : {
        'period' : 'Period 5',
        'start_time' : '2:10 PM',
        'end_time' : '3:10 PM',
    },
}

block_to_period_number = {
    'A' : [1,0,2,5,3,0,4,0],
    'B' : [2,5,3,0,4,0,1,0],
    'C' : [3,0,4,0,1,0,2,5],
    'D' : [4,0,1,0,2,5,3,0],
    'E' : [0,1,0,2,5,3,0,4],
    'F' : [0,2,5,3,0,4,0,1],
    'G' : [5,3,0,4,0,1,0,2],
    'H' : [0,4,0,1,0,2,5,3],
    'LN' : [6,6,6,6,6,6,6,6], # 6 is lunch
    '2' : [7,7,7,7,7,7,7,7], # 7 is special
    'Health10E' : [1,0,0,0,0,0,4,0],
}

def get_period_block(day: int, block: str, grade:int):

    if block == '' or block.strip() == "-":
        return None
    try:
        if block == "A-D17":
            block = "Health10E"
        else:
            block = block.split(":")[1].strip()
    except:
        raise Exception(f"{block}")
    if "*" in block:
        block = block.split("*")[0].capitalize()
        
    period = block_to_period_number[block][day-1]

    if period == 0:
        return None

    if period == 6:
        if grade < 11:
            return {
                'period' : 'First Lunch',
                'start_time' : f'{"12:15 PM" if day % 2 == 0 else "12:00 PM"}',
                'end_time' : '12:30 PM',
            }
        else:
            return {
                'period' : 'Second Lunch',
                'start_time' : "12:30 PM",
                'end_time' : '1:00 PM',
            }
    elif period == 7:
        return {
            'period' : 'Special',
            'start_time' : f'{"9:35" if day % 2 == 0 else "9:20"} AM',
            'end_time' : '9:55 AM',
        }
    else: 
        return period_times[f"Period {period} {'Even' if day % 2 == 0 else 'Odd'}"]

In [8]:
import pandas as pd

# Read in the data
df = pd.read_csv('Student Project CSV Jan 11-part 2.csv')

# drop nulls
df = df.fillna('')

# iterate through df

classes_to_look_for = [
    "US Study Hall",
    "Support Services",
    "US Advisory",
    "US Health Education",
    "CC College Counseling",
]

periods_to_add = {}

current_student = ''

for index, row in df.iterrows():
    grade = row['STUDENT: Grade']

    if row['STUDENT: Email 1'] != current_student:
        current_student = row['STUDENT: Email 1']

    if row["Subject"] in classes_to_look_for:
        try:
            meeting_times = {
                "Day 1": get_period_block(1, row["Day 1"], grade),
                "Day 2": get_period_block(2, row["Day 2"], grade),
                "Day 3": get_period_block(3, row["Day 3"], grade),
                "Day 4": get_period_block(4, row["Day 4"], grade),
                "Day 5": get_period_block(5, row["Day 5"], grade),
                "Day 6": get_period_block(6, row["Day 6"], grade),
                "Day 7": get_period_block(7, row["Day 7"], grade),
                "Day 8": get_period_block(8, row["Day 8"], grade),
            }
        except:
            print(row)
            raise

        period = {
            "course" : {
                "name" : row["Class"],
                "teacher" : row["Teacher"],
                "upper_or_lower" : "US",
            },
            "meeting_times" : meeting_times,
        }
        if current_student not in periods_to_add:
            periods_to_add[current_student] = [period]
        else:
            periods_to_add[current_student].append(period)

#print(periods_to_add)

In [9]:
# writing new periods over old data
import json

with open('schedules copy.json') as f:
    schedules = json.load(f)

for student in periods_to_add:
    new_courses = periods_to_add[student]
    student_schedule = schedules[student]

    for course in new_courses:
        course_info = course['course']

        meeting_times= course['meeting_times']
        for day in meeting_times:
            if meeting_times[day] is not None:

                period = meeting_times[day]['period']
                
                course_to_add = {
                    'course' : course_info,
                    'start_time': meeting_times[day]['start_time'],
                    'end_time' : meeting_times[day]['end_time'],
                    'period' : period,
                }

                existing_periods = student_schedule['days'][int(day.split(" ")[1])-1]['periods']
    
                for existing_period in existing_periods:
                    if existing_period['period'] == period:
                        print('True')
                        schedules[student]['days'][int(day.split(" ")[1])-1]['periods'].remove(existing_period)

                schedules[student]['days'][int(day.split(" ")[1])-1]['periods'].append(course_to_add)

# write new file to schedules.json
with open ('schedules.json', 'w') as f:
    json.dump(schedules, f, indent=4)

        

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
